# Sentence Embeddings

* get all sentence embeddings
* train network/ fine tune roberta model
* create a cheeky network!
* PageRank?

In [57]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import networkx as nx

In [32]:
# open dataframe
complete = pd.read_csv('../StarTrekNextGenScriptData/complete_data.csv')

# get sentences
sentences = list(complete['quote'])

# preview
sentences[:5]

["   You know, Morn -- there's nothing    quite as invigorating as breakfast    in a bar. Where else can you get    raw slug livers first thing in the    morning?",
 "   What's this?",
 '   What do you mean, "what\'s this?"    It\'s puree of beetle.',
 "   I didn't order it.",
 '   Of course you "didn\'t order it" --    you don\'t need to order it. You    have it after work every morning.']

In [18]:
# load my girl roberta
model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')

In [ ]:
# start by completing this problem on a mini model
baby = sentences[:100]
embeddings = model.encode(baby) # TBD later: embeddings = model.encode(sentences)

In [49]:
embedding_df = pd.DataFrame(embeddings)

In [51]:
pairwise_sim = pd.DataFrame(cosine_similarity(embedding_df))
pairwise_sim

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,1.000000,0.130096,0.175844,0.157960,0.246214,0.411002,0.435851,0.484361,0.432827,0.101364,...,0.114800,0.077198,0.160782,0.033112,0.206859,0.133362,0.189859,0.155083,0.197817,0.114290
1,0.130096,1.000000,0.284866,0.119260,0.032427,0.058398,0.139175,0.165895,0.005889,0.099763,...,0.157064,0.056267,0.023325,0.063735,0.093421,0.164814,0.116653,0.166354,0.105948,0.193397
2,0.175844,0.284866,1.000000,0.166654,0.161237,0.164123,0.356204,0.295577,0.176113,0.037612,...,0.092228,0.061688,0.045487,0.018874,0.270711,0.119516,0.155818,0.014401,0.117634,0.109572
3,0.157960,0.119260,0.166654,1.000000,0.757453,0.160583,0.156926,0.215654,0.159354,0.058400,...,0.167380,0.065887,0.031741,0.121495,0.106216,0.157213,0.159476,0.076278,0.268339,0.232927
4,0.246214,0.032427,0.161237,0.757453,1.000000,0.244612,0.224277,0.278436,0.264019,-0.028724,...,0.104641,0.061526,-0.025175,0.117872,0.097116,0.175673,0.168130,0.040509,0.266078,0.135338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.133362,0.164814,0.119516,0.157213,0.175673,0.162410,0.028124,0.135274,0.121789,0.262358,...,0.317083,0.184069,0.067881,0.117003,0.135221,1.000000,0.288549,0.244562,0.295216,0.216417
96,0.189859,0.116653,0.155818,0.159476,0.168130,0.139187,0.044261,0.112944,0.121836,0.107854,...,0.210087,0.245418,0.196721,0.183295,0.136496,0.288549,1.000000,0.301472,0.416906,0.311676
97,0.155083,0.166354,0.014401,0.076278,0.040509,0.128160,0.025423,0.158336,0.086065,0.182511,...,0.263865,0.383306,0.101596,0.062423,0.124320,0.244562,0.301472,1.000000,0.315372,0.462494
98,0.197817,0.105948,0.117634,0.268339,0.266078,0.203329,0.121883,0.172504,0.051579,0.138027,...,0.138189,0.170110,0.092808,0.269598,0.265692,0.295216,0.416906,0.315372,1.000000,0.469300


In [ ]:
# calculate all of the stats on similarity

all_similarities = pairwise_sim.values.flatten().tolist()
print('mean:',np.mean(all_similarities))
print('median:',np.median(all_similarities))
print('quartiles:',np.percentile(all_similarities,25),np.percentile(all_similarities,50),np.percentile(all_similarities,75))
print('min & max:',np.min(all_similarities),np.max(all_similarities))

mean: 0.13989120711694705
median: 0.12085149437189102
quartiles: 0.0624198317527771 0.12085149437189102 0.18736151978373528
min & max: -0.13194957375526428 1.000000238418579


In [ ]:
# lets filter by 75% quartile 
quart = np.percentile(all_similarities,75)

In [76]:
pairwise_sim[8][9]

0.070216924

In [77]:
g = nx.Graph()

for x in pairwise_sim.columns:
    for y in pairwise_sim.columns:
        if y <= x:
            continue
        value = pairwise_sim[x][y]
        if value >= quart:
            g.add_edge(x,y,weight=value)

 stuff below is tbd

In [ ]:
# create df with index, quote, embeddings

embeddings_df = pd.DataFrame({'quote':baby,'embeddings':list(embeddings)}).reset_index()
embeddings_df.head()

,index,quote,embeddings
0,0,"You know, Morn -- there's nothing quite ...","[-0.014900208, -0.071976624, -0.009477574, -0...."
1,1,What's this?,"[-0.0071644243, -0.013978246, -0.0136519885, 0..."
2,2,"What do you mean, ""what's this?"" It's pu...","[-0.049197827, -0.035305783, 0.00814294, 0.088..."
3,3,I didn't order it.,"[-0.043590456, -0.021896439, -0.01940514, 0.02..."
4,4,"Of course you ""didn't order it"" -- you d...","[-0.053526014, 0.0012190586, 0.0011013808, 0.0..."
...,...,...,...
95,95,It's the least I can do after...,"[-0.04241103, -0.0022942515, -0.011721256, 0.0..."
96,96,"It's all right, Julian. I forgive you. ...","[-0.039677955, -0.060667247, 0.009102529, 0.00..."
97,97,I'll be here. I guess it's not a sur...,"[0.056505203, -0.06760052, -0.04946052, -0.021..."
98,98,I can't believe I did that. I a...,"[-0.024098104, -0.06581943, -0.029327968, 0.02..."


In [42]:
# calculate pairwise cosine similarity! as recommended in sentence transformers
list(embeddings)

[array([-1.49002085e-02, -7.19766244e-02, -9.47757438e-03, -2.90310569e-02,
         1.41875800e-02,  1.63937416e-02,  2.76805907e-02, -3.72905619e-02,
        -6.35843817e-03, -4.82633477e-03,  3.46322097e-02, -2.68303268e-02,
         2.87671629e-02, -3.02961748e-02, -8.96742940e-02,  1.14269845e-01,
        -4.94523868e-02, -8.91323537e-02,  8.95914249e-03, -4.70567383e-02,
        -1.46471849e-02,  6.77025914e-02,  7.81802554e-03, -9.72901378e-03,
         3.02597438e-03,  3.96646671e-02,  3.83275300e-02,  2.67764460e-03,
         2.03774069e-02, -2.41375715e-02,  5.15765324e-03, -1.35345235e-02,
        -1.30845897e-03,  2.01618876e-02,  3.77856120e-02,  2.41360962e-02,
        -3.09586655e-02, -1.04861222e-02,  3.51767950e-02,  2.87077297e-02,
         8.27053934e-03, -5.93182482e-02, -4.71299924e-02,  5.08786924e-03,
         1.98854450e-02,  3.29294452e-03,  4.26044986e-02, -2.72859409e-02,
        -6.97635068e-03,  2.26252228e-02,  1.28249861e-02,  4.23678849e-03,
        -2.6